<a href="https://colab.research.google.com/github/bidhan7/Data-Processing-Visualization-/blob/main/Big_Query_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bidhan Subedi
## Data Visiualization



## SQL with BigQuery
The first thing, most of us already have an account at Cloud Resource Manager if you already have gmail logged in your computer. I required to do each of the following steps successfully in getting this document to work.  

## Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Now that I am authenticated, I can start to playing around with the dataset.  I am going to look at the bikers trip data from Austin. Yes, I have a project called `data-visualization-338723` in my BigQuery interface.

## We will use SQL to compute from dataset

`SELECT` picks the columns and `FROM` picks the table you need. 

### Computing the no of entries in the dataset.

In [ ]:
%%bigquery --project data-visualization-338723
SELECT 
  COUNT(*) as total_entries
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_entries
0,1424786


That's is a lot of entries.

As there is alot of entries in the data set excel will have hard time computing. SQL will do the data manipulations on the database server much better.

 

### Computing The Longest Trip

In [ ]:
%%bigquery --project data-visualization-338723
SELECT 
  MAX(duration_minutes) as Longest_trip_in_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,Longest_trip_in_minutes
0,34238


The longest trip was 34238 minutes!

### Computing the average time for a trip based on starting point

In [ ]:
%%bigquery --project data-visualization-338723
SELECT 
  start_station_name, 
  AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name is not null
GROUP BY start_station_name

,start_station_name,Average_time_for_trip
0,Zilker Park West,27.168259
1,Toomey Rd @ South Lamar,28.115798
2,State Capitol @ 14th & Colorado,30.421842
3,Waller & 6th St.,23.265843
4,Pease Park,28.648889
...,...,...
188,Republic Square @ Guadalupe & 4th St.,23.067313
189,3rd & West,19.937466
190,3rd/West,34.152547
191,Nueces & 26th,16.805353


Last column on right side of the table represents average time for every indiviual trip.

#### This is just me playing around .

In [ ]:
%%bigquery --project data-visualization-338723
SELECT 
  COUNT(start_station_name) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_trips
0,1424786


### Computing how many trips start at each starting point

In [ ]:
%%bigquery --project data-visualization-338723
SELECT 
  start_station_name,
  COUNT(*) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,start_station_name,total_trips
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758
...,...,...
188,Marketing Event,4
189,Eeyore's 2018,2
190,Stolen,1
191,Eeyore's 2017,1


### JUST PLAYING AROUND


In [ ]:
%%bigquery --project data-visualization-338723

SELECT *, (CASE WHEN start_station_name = end_station_name THEN 1 ELSE 0 END) as round_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 200


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes,round_trips
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39,0
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,1
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,1
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30,1
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19,1
...,...,...,...,...,...,...,...,...,...,...
195,9900292320,Local365,866,2015-10-03 12:12:44+00:00,2576,Rainey @ River St,2576,Rainey @ River St,1,1
196,9900292347,Local365,306,2015-10-03 20:12:04+00:00,1006,Zilker Park West,1006,Zilker Park West,0,1
197,9900292363,Local365,735,2015-10-03 12:12:00+00:00,1008,Nueces @ 3rd,2712,Toomey Rd @ South Lamar,8,0
198,9900292365,Local365,409,2015-10-03 12:12:55+00:00,1008,Nueces @ 3rd,1007,Lavaca & 6th,0,0


I combined the table above and found the total number of round trips which has same starting and ending station. 

In [ ]:
%%bigquery --project data-visualization-338723
SELECT start_station_name, SUM(round_trips) as number_of_round_trips
FROM (
  SELECT *, (CASE WHEN start_station_name = end_station_name THEN 1 ELSE 0 END) as round_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
)
GROUP BY start_station_name,end_station_name
ORDER BY number_of_round_trips DESC
LIMIT 200

,start_station_name,number_of_round_trips
0,21st & Speedway @PCL,13109
1,Riverside @ S. Lamar,12302
2,Zilker Park,9754
3,Rainey St @ Cummings,8676
4,2nd & Congress,5830
...,...,...
195,East 11th St. at Victory Grill,0
196,East 2nd & Pedernales,0
197,East 11th St. at Victory Grill,0
198,Capital Metro HQ - East 5th at Broadway,0


### Computing the second most popular starting station

In [ ]:
%%bigquery --project data-visualization-338723

SELECT COUNT(*) as total_trips, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,total_trips,start_station_name
0,72799,21st & Speedway @PCL
1,40635,Riverside @ S. Lamar
2,36520,City Hall / Lavaca & 2nd
3,35307,2nd & Congress
4,34758,Rainey St @ Cummings
...,...,...
188,4,Marketing Event
189,2,Eeyore's 2018
190,1,Stolen
191,1,Eeyore's 2017


Above table shows total trips started from each stations.

In [ ]:
%%bigquery --project data-visualization-338723
SELECT MAX(most_pop_trip) as most_pop_trip
FROM (
SELECT COUNT(*) as most_pop_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
)

,most_pop_trip
0,72799


That's the first popular trip which was made 72799 time from starting station. Below i will find second most popular trip.

In [ ]:
%%bigquery --project data-visualization-338723
SELECT start_station_name, COUNT(*) as most_pop_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
HAVING most_pop_trip!= (SELECT MAX(most_pop_trip)as max_trip
FROM (
SELECT COUNT(*) as most_pop_trip, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
))
ORDER BY most_pop_trip DESC
LIMIT 1

,start_station_name,most_pop_trip
0,Riverside @ S. Lamar,40635


### Computing the trips lasted over an hour and were a round trip (started and stopped at the same station)

In [ ]:
%%bigquery --project data-visualization-338723

SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name

,start_station_name,duration_minutes
0,Toomey Rd @ South Lamar,82
1,Waller & 6th St.,76
2,Waller & 6th St.,75
3,Waller & 6th St.,74
4,Waller & 6th St.,73
...,...,...
228334,3rd/West,63
228335,3rd & West,63
228336,3rd/West,63
228337,3rd/West,63


In [ ]:
%%bigquery --project data-visualization-338723

SELECT *, COUNTIF(duration_minutes>60) as number_of_trip_took_over_an_hour
FROM (
  SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
)
GROUP BY duration_minutes, start_station_name
Order BY number_of_trip_took_over_an_hour DESC

,start_station_name,duration_minutes,number_of_trip_took_over_an_hour
0,Riverside @ S. Lamar,62,109
1,Riverside @ S. Lamar,61,95
2,Riverside @ S. Lamar,63,94
3,Riverside @ S. Lamar,69,90
4,Zilker Park,61,88
...,...,...,...
24822,26th/Nueces,40,0
24823,26th/Nueces,17,0
24824,26th/Nueces,53,0
24825,26th/Nueces,48,0


In [ ]:
%%bigquery --project data-visualization-338723

SELECT station_id, property_type, AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`


,station_id,property_type
0,2576,None
1,2712,None
2,3381,None
3,3464,None
4,1001,None
5,1002,None
6,1003,None
7,1004,None
8,1005,None
9,1006,None


### Computing the average trip on Austin bike share grouping by property_type column from the bikeshare_stations table. 

### Is average time on trips starting from 'parkland' longer than from 'sidewalks'?

In [ ]:
%%bigquery --project data-visualization-338723

WITH t as(
SELECT name, property_type
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
)
SELECT t.property_type as stating_startion_type, 
    AVG (trips.duration_minutes) as average_min,
    COUNT(*) as number_of_trips,
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips
  JOIN t
  ON trips.start_station_name = t.name

GROUP BY t.property_type
ORDER by average_min

,stating_startion_type,average_min,number_of_trips
0,undetermined_parking,23.098028,225333
1,None,25.697646,68046
2,paid_parking,25.852535,280405
3,sidewalk,29.263569,432392
4,,33.502832,19776
5,parkland,39.375957,141806
6,nonmetered_parking,49.046326,11052


Yes, the average time on trips starting from 'Parkland' is 39.37 which is longer than from average time staring from sidewalks 29.27.

### Computing how many number of trips, "Weekender" subscriber type are there to cross over 30 min duration?

I have compared Weekender who does biking for more than 30 minutes with the total number of trips Weekender make.

In [ ]:
%%bigquery --project data-visualization-338723

SELECT subscriber_type,  COUNTIF(duration_minutes>30) as number_of_trip, COUNT(*) as total_no_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE subscriber_type='Weekender'
GROUP BY subscriber_type


,subscriber_type,number_of_trip,total_no_trip
0,Weekender,10143,45012
